In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import sys
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Get the directory of the current script
try:
    # This will work if the script is being run as a file
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Use the current working directory if __file__ is not defined
    current_dir = os.getcwd()
# Move up one directory level from EDA to the parent directory
parent_dir = os.path.dirname(current_dir)
# Construct the path to the xdrive folder
xdrive_path = os.path.join(parent_dir, 'xdrive')
# Add the xdrive path to sys.path
sys.path.append(xdrive_path)

featureeng_path = os.path.join(parent_dir, 'Feature Engineering')
# Add the xdrive path to sys.path
sys.path.append(featureeng_path)

# Construct the path to the xdrive folder
xdrive_path = os.path.join(parent_dir, 'Pipeline')
# Add the xdrive path to sys.path
sys.path.append(xdrive_path)
from build_a_pipeline import build_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TimeDistributed, Dense
from build_a_pipeline import build_pipeline_P0_foward_price


In [2]:

model_performance = []
#x_train, x_val, X_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(target_column = 'P0',drop_features=False)
x_train, x_val, X_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(drop_features=True, 
                                                                                remove_outliers = True, 
                                                                                percentage_of_outliers=0.01, 
                                                                                deal_with_skewness = True) #drop_features = False
# x_train = x_train.drop(columns = ['Average_Settlement_Price_SE/CW(MWh)', 'Min_Settlement_Price_SE(MWh)'])
# x_val = x_val.drop(columns = ['Average_Settlement_Price_SE/CW(MWh)', 'Min_Settlement_Price_SE(MWh)'])
# X_test = X_test.drop(columns = ['Average_Settlement_Price_SE/CW(MWh)', 'Min_Settlement_Price_SE(MWh)'])
datetime_index = x_val.index
# Convert x_train and x_val to NumPy arrays
x_val_original = x_val
X_test_original = X_test
x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
# Reshape x_train and x_val to have the required 3D shape
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])  # [samples, time_steps=1, features]
x_val = x_val.reshape(x_val.shape[0], 1, x_val.shape[1])    



C:\Users\veron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [12]:
from sklearn.metrics import r2_score
# Build the LSTM model
multivariate_lstm = Sequential([
    LSTM(195, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True),
    Dropout(0.3),
    TimeDistributed(Dense(95, activation='selu')), 
    Dropout(0.1),
    Flatten(),
    Dense(1)  # Output layer
])

# ModelCheckpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('multivariate_lstm.keras', monitor='val_loss', save_best_only=True)

# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Optimizer with adjusted learning rate
optimizer = Adam(learning_rate=6e-3, amsgrad=True)

# Loss function and metric
loss = tf.keras.losses.MeanSquaredError()
metric = [tf.keras.metrics.RootMeanSquaredError()]

# Compile the model
multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

# Train the model (make sure you pass both x_val and y_val)
history = multivariate_lstm.fit(x_train, y_train, epochs=120,
                                validation_data=(x_val, y_val),
                                callbacks=[early_stopping, model_checkpoint])

# Optionally, print the final RMSE
train_rmse = history.history['root_mean_squared_error'][-1]
val_rmse = history.history['val_root_mean_squared_error'][-1]
print(f"Final train RMSE: {train_rmse}")
print(f"Final validation RMSE: {val_rmse}")
model_performance.append({
    'Model': '1st Model',
    'LayerExtra': 'TimeDistributed, LSTM',
    'Train RMSE': train_rmse,
    'Validation RMSE': val_rmse
})
from tensorflow.keras.models import load_model
best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(x_val)
r_squared = r2_score(y_val, y_pred)
print(f"R-squared: {r_squared}")

Epoch 1/120


C:\Users\veron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 30350.6074 - root_mean_squared_error: 173.8955 - val_loss: 2197.4275 - val_root_mean_squared_error: 46.8767
Epoch 2/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11196.2676 - root_mean_squared_error: 105.2156 - val_loss: 11147.5977 - val_root_mean_squared_error: 105.5822
Epoch 3/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6691.0664 - root_mean_squared_error: 81.4461 - val_loss: 17691.9199 - val_root_mean_squared_error: 133.0110
Epoch 4/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2761.6912 - root_mean_squared_error: 52.2503 - val_loss: 16718.5254 - val_root_mean_squared_error: 129.3001
Epoch 5/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1236.3414 - root_mean_squared_error: 34.8854 - val_loss: 18933.7441 - val_root_mean_squared_error: 137.5999
Epoch 6/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1109.9252 - root_mean_squared_error: 33.2086 - val_loss: 16159.8057 - val_root_mean_squared_error: 127.1212
Epoch 7

Improved version

## Trying the pipeline with no feature selection dropping

In [ ]:

x_train, x_val, X_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(drop_features=False, 
                                                                                remove_outliers = True, 
                                                                                percentage_of_outliers=0.01, 
                                                                                train_size = 0.75, 
                                                                                val_size = 0.15, 
                                                                                deal_with_skewness = True) #drop_features = False

# Convert x_train and x_val to NumPy arrays
x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
# Reshape x_train and x_val to have the required 3D shape
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])  # [samples, time_steps=1, features]
x_val = x_val.reshape(x_val.shape[0], 1, x_val.shape[1])    

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [ ]:
from tensorflow.keras.initializers import GlorotUniform
# Build the LSTM model
multivariate_lstm = Sequential([
    LSTM(195, input_shape=(x_train.shape[1], x_train.shape[2]), kernel_initializer=GlorotUniform(seed=42), return_sequences=True),
    TimeDistributed(Dense(95, activation='selu', kernel_initializer=GlorotUniform(seed=42))), 
    Flatten(),
    Dense(1)  # Output layer
])

# ModelCheckpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('multivariate_lstm.keras', monitor='val_loss', save_best_only=True)

# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Optimizer with adjusted learning rate
optimizer = Adam(learning_rate=6e-3, amsgrad=True, clipnorm=1.0)
#optimizer = Adam(learning_rate=0.001, clipnorm=1.0)

# Loss function and metric
loss = tf.keras.losses.MeanSquaredError()
metric = [tf.keras.metrics.RootMeanSquaredError()]

# Compile the model
multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

# Train the model (make sure you pass both x_val and y_val)
history = multivariate_lstm.fit(x_train, y_train, epochs=120,
                                validation_data=(x_val, y_val),
                                callbacks=[early_stopping, model_checkpoint])

# Optionally, print the final RMSE
train_rmse = history.history['root_mean_squared_error'][-1]
val_rmse = history.history['val_root_mean_squared_error'][-1]
print(f"Final train RMSE: {train_rmse}")
print(f"Final validation RMSE: {val_rmse}")
model_performance.append({
    'Model': '1st Model',
    'LayerExtra': 'TimeDistributed, LSTM',
    'Train RMSE': train_rmse,
    'Validation RMSE': val_rmse
})

best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(x_val)
r_squared = r2_score(y_val, y_pred)
print(f"R-squared: {r_squared}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TimeDistributed, Dense

# Build the LSTM multivariate_lstm
multivariate_lstm = Sequential([
    LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True),
    TimeDistributed(Dense(100, activation='relu')), 
    Flatten(),
    Dense(1)  # Output layer
])

# ModelCheckpoint to save the best multivariate_lstm based on validation loss
model_checkpoint = ModelCheckpoint('multivariate_lstm.keras', monitor='val_loss', save_best_only=True)

# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Optimizer with adjusted learning rate
optimizer = Adam(learning_rate=6e-3, amsgrad=True)

# Loss function and metric
loss = tf.keras.losses.MeanSquaredError()
metric = [tf.keras.metrics.RootMeanSquaredError()]

# Compile the multivariate_lstm
multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

# Train the multivariate_lstm (make sure you pass both x_val and y_val)
history = multivariate_lstm.fit(x_train, y_train, epochs=120,
                                validation_data=(x_val, y_val),
                                callbacks=[early_stopping, model_checkpoint])

# Optionally, print the final RMSE
train_rmse = history.history['root_mean_squared_error'][-1]
val_rmse = history.history['val_root_mean_squared_error'][-1]
print(f"Final train RMSE: {train_rmse}")
print(f"Final validation RMSE: {val_rmse}")

model_performance.append({
    'multivariate_lstm': multivariate_lstm,
    'Train RMSE': train_rmse,
    'Validation RMSE': val_rmse
})

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(x_val)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

y_pred = y_pred.flatten()
predictions_df = pd.DataFrame(y_pred, index=datetime_index, columns=['Predicted'])
predictions_df = predictions_df.sort_index()
y_val = y_val.sort_index()

# Plotting y_actual vs y_pred
plt.figure(figsize=(12, 6))
plt.plot(y_val.index, y_val, label='Actual', marker='o')
plt.plot(y_val.index, predictions_df['Predicted'], label='Predicted', marker='x')
plt.title('Actual vs Predicted Values MultiVariateLSTM')
plt.xlabel('DateTime')
plt.ylabel('Values')

# Rotate the x-axis labels for better visibility
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Assuming `df` is the DataFrame with 42 columns and `x_val_original` contains 'Average_Settlement_Price_SE/CW(MWh)'
# Create the figure
fig = go.Figure()

# Plot the columns from df
for column in predictions_df.columns:
    fig.add_trace(go.Scatter(x=predictions_df.index, y=predictions_df[column], mode='lines', name=column))

fig.add_trace(go.Scatter(x=y_val.index, y=y_val, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=x_val_original.index, y=x_val_original['Average_Settlement_Price_SE/CW(MWh)'], mode='lines', name='Average_Settlement_Price_SE/CW(MWh)'))

# Plot the 'Average_Settlement_Price_SE/CW(MWh)' from x_val_original with a secondary y-axis
fig.add_trace(go.Scatter(x=x_val_original.index, 
                         y=x_val_original['Hydro_Inflow_NE(MWavg)'], 
                         mode='lines', 
                         name='Hydro_Inflow_NE(MWavg)', 
                         yaxis='y2'))  # Use the second y-axis

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Visualization of All Columns in the Dataset',
    xaxis_title="DateTime or Index",
    yaxis_title="Values (df)",
    yaxis2=dict(
        title="Average Settlement Price (x_val_original)",
        overlaying="y",  # Overlay the second y-axis on the primary y-axis
        side="right",  # Place the secondary y-axis on the right
        showgrid=False  # Optionally hide the grid for the secondary y-axis
    ),
    xaxis_tickangle=45,  # Rotate x-axis labels for better visibility
    template="plotly_dark",  # Optional: change the theme
    width=1200,  # Optional: adjust plot width
    height=600,  # Optional: adjust plot height
)

# Show the interactive plot
fig.show()


## FINAL TESTING
- training on val and train
- testing on test

In [ ]:

x_train, x_val, x_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(drop_features=False, 
                                                                                remove_outliers = True, 
                                                                                percentage_of_outliers=0.01, 
                                                                                train_size = 0.75, 
                                                                                val_size = 0.15,
                                                                                 deal_with_skewness = True) #drop_features = False
datetime_index = x_test.index
#save these values for later
X_test_last= X_test.sort_index()
X_test_last = X_test_last.tail(10)
datetime_index_final = X_test_last.index
X_test_last = X_test_last.to_numpy()  # Convert DataFrame to NumPy array
X_test_last = X_test_last.reshape(X_test_last.shape[0], 1, X_test_last.shape[1])
y_test_last = y_test.sort_index()
y_test_last = y_test_last.tail(10)

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

# merge train and validation
x_train_combined = np.vstack([x_train, x_val])  # Combine x_train and x_val
y_train_combined = np.concatenate([y_train, y_val])  # Combine y_train and y_val

#reshape
x_train = x_train_combined.reshape(x_train_combined.shape[0], 1, x_train_combined.shape[1])  # [samples, time_steps=1, features]
# Test data
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1]) 

In [ ]:
from tensorflow.keras.initializers import GlorotUniform
# Build the LSTM model
multivariate_lstm = Sequential([
    LSTM(105, input_shape=(x_train.shape[1], x_train.shape[2]), kernel_initializer=GlorotUniform(seed=42), return_sequences=True),
    Dropout(0.4),
    TimeDistributed(Dense(75, activation='selu', kernel_initializer=GlorotUniform(seed=42))), 
    Dropout(0.2),
    Flatten(),
    Dense(1)  # Output layer
])

# ModelCheckpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('multivariate_lstm.keras', monitor='val_loss', save_best_only=True)

# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Optimizer with adjusted learning rate
optimizer = Adam(learning_rate=6e-3, amsgrad=True, clipnorm=1.0)
#optimizer = Adam(learning_rate=0.001, clipnorm=1.0)

# Loss function and metric
loss = tf.keras.losses.MeanSquaredError()
metric = [tf.keras.metrics.RootMeanSquaredError()]

# Compile the model
multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

history = multivariate_lstm.fit(x_train, y_train_combined, epochs=120,
                                validation_data=(x_test, y_test),
                                callbacks=[early_stopping, model_checkpoint])

# Optionally, print the final RMSE
train_rmse = history.history['root_mean_squared_error'][-1]
val_rmse = history.history['val_root_mean_squared_error'][-1]
print(f"Final train RMSE: {train_rmse}")
print(f"Final test RMSE: {val_rmse}")
model_performance.append({
    'Model': '1st Model',
    'LayerExtra': 'TimeDistributed, LSTM',
    'Train RMSE': train_rmse,
    'Validation RMSE': val_rmse
})

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

y_pred = y_pred.flatten()
predictions_df = pd.DataFrame(y_pred, index=datetime_index, columns=['Predicted'])
y_test_df = pd.DataFrame(y_test, index=datetime_index, columns=['Actual'])
predictions_df = predictions_df.sort_index()
y_test_df = y_test_df.sort_index()

# Plotting y_actual vs y_pred
plt.figure(figsize=(12, 6))
plt.plot(predictions_df.index, y_test_df['Actual'], label='Actual', marker='o')
plt.plot(y_test_df.index, predictions_df['Predicted'], label='Predicted', marker='x')
plt.title('Actual vs Predicted Values MultiVariateLSTM on TEST')
plt.xlabel('DateTime')
plt.ylabel('Values')

# Rotate the x-axis labels for better visibility
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

Last 10 values prediction

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(X_test_last)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
y_pred = y_pred.flatten()
predictions_df = pd.DataFrame(y_pred, index=datetime_index_final, columns=['Predicted'])
predictions_df = predictions_df.sort_index()

# Plotting y_actual vs y_pred
plt.figure(figsize=(12, 6))
plt.plot(y_test_last.index, y_test_last, label='Actual', marker='o')
plt.plot(predictions_df.index, predictions_df['Predicted'], label='Predicted', marker='x')
plt.title('Actual vs Predicted Values MultiVariateLSTM on TEST')
plt.xlabel('DateTime')
plt.ylabel('Values')

# Rotate the x-axis labels for better visibility
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

This code is not performing on September or October 2024, but lets look at beginning of the year

In [ ]:
#x_train, x_val, x_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(target_column='P0', drop_features=False)
x_train, x_val, x_test, y_train, y_val, y_test = build_pipeline_P0_foward_price(drop_features=False, 
                                                                                remove_outliers = True, 
                                                                                percentage_of_outliers=0.05, 
                                                                                train_size = 0.70, 
                                                                                val_size = 0.05) #drop_features = False
x_test.index = pd.to_datetime(x_test.index)
y_test.index = pd.to_datetime(y_test.index)
datetime_index = x_test.index
#save these values for later
X_test_last= X_test.sort_index()
X_test_last = x_test.loc['2024-02-01':'2024-05-30']
datetime_index_final = X_test_last.index
X_test_last = X_test_last.to_numpy()  # Convert DataFrame to NumPy array
X_test_last = X_test_last.reshape(X_test_last.shape[0], 1, X_test_last.shape[1])
y_test_last = y_test.sort_index()
y_test_last = y_test_last.loc['2024-02-01':'2024-05-30']

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

# merge train and validation
x_train_combined = np.vstack([x_train, x_val])  # Combine x_train and x_val
y_train_combined = np.concatenate([y_train, y_val])  # Combine y_train and y_val

#reshape
x_train = x_train_combined.reshape(x_train_combined.shape[0], 1, x_train_combined.shape[1])  # [samples, time_steps=1, features]
# Test data
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1]) 

In [ ]:
from tensorflow.keras.initializers import GlorotUniform
# Build the LSTM model
multivariate_lstm = Sequential([
    LSTM(250, input_shape=(x_train.shape[1], x_train.shape[2]), kernel_initializer=GlorotUniform(seed=42), return_sequences=True),
    Dropout(0.4),
    TimeDistributed(Dense(200, activation='selu', kernel_initializer=GlorotUniform(seed=42))), 
    Dropout(0.2),
    TimeDistributed(Dense(200, activation='selu', kernel_initializer=GlorotUniform(seed=42))), 
    Dropout(0.2),
    TimeDistributed(Dense(100, activation='selu', kernel_initializer=GlorotUniform(seed=42))),
    Flatten(),
    Dense(1)  # Output layer
])

# ModelCheckpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('multivariate_lstm.keras', monitor='val_loss', save_best_only=True)

# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Optimizer with adjusted learning rate
optimizer = Adam(learning_rate=6e-3, amsgrad=True, clipnorm=1.0)
#optimizer = Adam(learning_rate=0.001, clipnorm=1.0)

# Loss function and metric
loss = tf.keras.losses.MeanSquaredError()
metric = [tf.keras.metrics.RootMeanSquaredError()]

# Compile the model
multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

history = multivariate_lstm.fit(x_train, y_train_combined, epochs=120,
                                validation_data=(x_test, y_test),
                                callbacks=[early_stopping, model_checkpoint])

# Optionally, print the final RMSE
train_rmse = history.history['root_mean_squared_error'][-1]
val_rmse = history.history['val_root_mean_squared_error'][-1]
print(f"Final train RMSE: {train_rmse}")
print(f"Final test RMSE: {val_rmse}")
model_performance.append({
    'Model': '1st Model',
    'LayerExtra': 'TimeDistributed, LSTM',
    'Train RMSE': train_rmse,
    'Validation RMSE': val_rmse
})

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('multivariate_lstm.keras')
y_pred = best_model.predict(X_test_last)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

y_pred = y_pred.flatten()
predictions_df = pd.DataFrame(y_pred, index=datetime_index_final, columns=['Predicted'])
y_test_df = pd.DataFrame(y_test_last, index=datetime_index_final, columns=['Actual'])
predictions_df = predictions_df.sort_index()
y_test_df = y_test_df.sort_index()

# Plotting y_actual vs y_pred
plt.figure(figsize=(12, 6))
plt.plot(datetime_index_final, y_test_last, label='Actual', marker='o')
plt.plot(y_test_df.index, predictions_df['Predicted'], label='Predicted', marker='x')
plt.title('Actual vs Predicted Values MultiVariateLSTM on TEST')
plt.xlabel('DateTime')
plt.ylabel('Values')

# Rotate the x-axis labels for better visibility
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()